In [ ]:
from scipy._lib._test_ccallback import test_get_data_capsule

from models.graph_transformers.SAT.sat import models
from models.graph_transformers.SAT.sat import layers
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


def ptr_to_complete_edge_index(ptr):
    from_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat_interleave(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    to_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    combined_complete_edge_index = torch.vstack((torch.cat(from_lists, dim=0), torch.cat(to_lists, dim=0)))
    return combined_complete_edge_index
test_data = torch.FloatTensor([1., 2., 3., 4., 5.]).unsqueeze(1)
edge_index = torch.LongTensor([[0,1,2],[1, 2, 3]])

test_graph = Data(x=test_data, edge_index=edge_index)
test_data2 = torch.FloatTensor([-1., -2., -3., -4., 10]).unsqueeze(1)
edge_index2 = torch.LongTensor([[0,1],[1, 2]])
test_graph2 = Data(x=test_data2, edge_index=edge_index2)
loader = iter(DataLoader([test_graph, test_graph2], batch_size=2))
batch = next(loader)


batch.complete_edge_index = torch.LongTensor([[0,1,5],[1,2,6]])
# batch.complete_edge_index = ptr_to_complete_edge_index(batch.ptr)


model = models.GraphTransformer(in_size=1,
                                num_class=1,
                                in_embed=False,
                                d_model=3,
                                dim_feedforward=2,
                                num_heads=1,
                                num_layers=1,
                                abs_pe=False,
                                abs_pe_dim=None,

                                use_edge_attr=False,
                                num_edge_features=0,
                                dropout=0.,
                                k_hop=2)


#model(batch)

In [ ]:
amrlayer = layers.AttentionAMR(1, 0, 1, 0, True).cuda()

In [ ]:
batch.ptr

In [ ]:
edge_index_source = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[0][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()
edge_index_target = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[1][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()

amrlayer(batch.x.cuda(), batch.x.cuda(), batch.edge_index.cuda(), edge_index_source, edge_index_target, ptr=batch.ptr.cuda())

In [ ]:
embed = torch.nn.Linear(1000, 128)

In [ ]:
import pickle
with open("data/hol4/data/torch_graph_dict.pk", "rb") as f:
    tg = pickle.load(f)
test_graph_1 = tg[list(tg.keys())[0]]
test_graph_1
test_graph_2 = tg[list(tg.keys())[2]]
test_graph_2
loader = iter(DataLoader([test_graph_1, test_graph_2], batch_size=2))
batch = next(loader)

In [ ]:

from models.graph_transformers.SAT.sat import layers
amrlayer = layers.AttentionAMR(128,0,2,0,True)

x = embed(batch.x)

out_s, out_t = amrlayer(x, x, batch.edge_index)

In [ ]:
amrlayer(out_s, out_t, batch.edge_index)

In [ ]:
batch.edge_index

In [ ]:

batch.edge_attr = batch.edge_attr.long()
model = models.AMRTransformer(in_size=1000,num_class=0,d_model=128,num_heads=4,dim_feedforward=128,dropout=0.1,use_edge_attr=True,num_layers=1,layer_norm=True,in_embed=False).cuda()
model(batch.cuda())

In [ ]:
torch.tensor([1.,2.,3.,4.,5.]).to(torch.tensor([1.,1.,2.,5,5234,643,2345]))

In [ ]:
batch.x

In [ ]:
import torch
from models.graph_transformers.SAT.sat.layers import AttentionRelations
model = AttentionRelations(embed_dim=8, edge_dim=32)
x = torch.rand(11, 8)
edge_index = torch.LongTensor([[0,1,2,3,4,9,8,7,6,8],[4,3,2,1,0,7,8,8,6,6]])
edge_attr = torch.LongTensor([0,0,1,3,0,1,2,3,2,1])
softmax_idx = [5]
model(x, edge_index, softmax_idx, edge_attr)

In [ ]:
torch.tensor_split(x, [0,5,10])

In [ ]:
'''
Test for dropout memory issues using TransformerEncoderLayer
'''

In [ ]:
from models.transformer_encoder_model import TransformerEmbedding


In [1]:
import torch
# using standard TransformerEncoderLayer API, get very large memory increase when using dropout
test_input = torch.rand((1024,32,128)).cuda()

encoder_layers = torch.nn.TransformerEncoderLayer(128, 4, 8, dropout=0.1)
test_model = torch.nn.TransformerEncoder(encoder_layers, 8).cuda()
test_model = torch.compile(test_model)

op = torch.optim.Adam(test_model.parameters())
op.zero_grad()

out = test_model(test_input)

loss = torch.sum(out)

loss.backward()
op.step()

In [ ]:
import pymongo
import torch

In [ ]:
class DatabaseDataset(torch.utils.data.Dataset):
    def __init__(self, query, batch_size, collection: str, database: str):
        self.batch_size = batch_size

        client = pymongo.MongoClient()
        self.db = client[database]
        self.collection = self.db[collection]
        self.query = query

        # Or non-approximate method, if the approximate method
        # returns smaller number of items you should be fine
        # self.length = self.db.estimated_document_count()

        self.cursor = None

    def __enter__(self):
        # Ensure that this find returns the same order of query every time
        # If not, you might get duplicated data
        # It is rather unlikely (depending on batch size), shouldn't be a problem
        # for 20 million samples anyway
        self.cursor = self.db.find(self.query)
        return self

    def shuffle(self):
        # Find a way to shuffle data so it is returned in different order
        # If that happens out of the box you might be fine without it actually
        pass

    def __exit__(self, *_, **__):
        # Or anything else how to close the connection
        self.cursor.close()

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, index):
        # Read takes long, hence if you can load a batch of documents it should speed things up
        examples = self.cursor[index * self.batch_size : (index + 1) * self.batch_size]

        # Do something with this data
        ...
        # Return the whole batch
        return data, labels

In [ ]:
train_cursor = split_collection.aggregate([{"$match": {"split": "train"}}, {"$sample": {"size": 10000000}}])
batches = get_batches(train_cursor, self.batch_size)


In [ ]:
from utils.mongodb_utils import get_batches
from torch_geometric.data import Batch
from pymongo import MongoClient
import torch
from torch.utils.data import DataLoader
import lightning.pytorch as pl

In [ ]:
class MyIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, cursor, buf_size):
        super(MyIterableDataset).__init__()
        self.cursor = cursor
        self.batches = get_batches(self.cursor, batch_size=buf_size)
        self.curr_batches = next(self.batches)
        self.remaining = len(self.curr_batches)

    def __iter__(self):
        return self

    def __next__(self):
        if self.remaining == 0:
            self.curr_batches = next(self.batches)
            self.remaining = len(self.curr_batches)

        self.remaining -= 1

        if self.remaining >= 0:
            return self.curr_batches.pop()
        else:
            raise StopIteration




class PremiseSelectionSeparateGraphs(pl.LightningDataModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        client = MongoClient()
        self.db = client[self.config['db_name']]
        self.collection = self.db[self.config['collection_name']]
        self.batch_size = self.config['batch_size']
        self.options = config['options']

    def sample_to_link(self, sample):
        options = self.options
        stmt_graph = sample['stmt_graph']
        conj_graph = sample['conj_graph']
        y = sample['y']

        x1 = conj_graph['onehot']
        x1_mat = torch.LongTensor(x1)

        x2 = stmt_graph['onehot']
        x2_mat = torch.LongTensor(x2)

        ret = LinkData(x_s=x2_mat, x_t=x1_mat, y=torch.tensor(y))

        if 'edge_index' in options:
            if 'edge_index' in conj_graph and 'edge_index' in stmt_graph:
                x1_edge_index = conj_graph['edge_index']
                x1_edge_index = torch.LongTensor(x1_edge_index)

                x2_edge_index = stmt_graph['edge_index']
                x2_edge_index = torch.LongTensor(x2_edge_index)

                ret.edge_index_t = x1_edge_index
                ret.edge_index_s = x2_edge_index
            else:
                raise NotImplementedError

        if 'edge_attr' in options:
            if 'edge_attr' in conj_graph and 'edge_attr' in stmt_graph:
                x1_edge_attr = conj_graph['edge_attr']
                x1_edge_attr = torch.LongTensor(x1_edge_attr)

                x2_edge_attr = stmt_graph['edge_attr']
                x2_edge_attr = torch.LongTensor(x2_edge_attr)

                ret.edge_attr_t = x1_edge_attr
                ret.edge_attr_s = x2_edge_attr
            else:
                raise NotImplementedError

        # Edge index used to determine where attention is propagated in Message Passing Attention schemes

        if 'attention_edge_index' in options:
            if 'attention_edge_index' in conj_graph and 'attention_edge_index' in stmt_graph:
                ret.attention_edge_index_t = conj_graph['attention_edge_index']
                ret.attention_edge_index_s = stmt_graph['attention_edge_index']
            else:
                # Default is global attention
                ret.attention_edge_index_t = torch.cartesian_prod(torch.arange(x1_mat.size(0)),
                                                                  torch.arange(x1_mat.size(0))).transpose(0, 1)

                ret.attention_edge_index_s = torch.cartesian_prod(torch.arange(x2_mat.size(0)),
                                                                  torch.arange(x2_mat.size(0))).transpose(0, 1)

        # todo make data options have possible values i.e. options['softmax_idx'] == AMR, use edges, else directed attention etc.
        if 'softmax_idx' in options:
            ret.softmax_idx_t = x1_edge_index.size(1)
            ret.softmax_idx_s = x2_edge_index.size(1)

        return ret

    def custom_collate(self, data):
        data_list = [self.sample_to_link(d) for d in data]
        return Batch.from_data_list(data_list, follow_batch=['x_s', 'x_t'])
        # return [self.sample_to_link(d) for d in data]

    def setup(self, stage: str):
        if stage == "fit":
            # num_train = self.collection.count_documents
            # print (self.collection.find_one())
            # self.train_cursor = self.collection.aggregate([{"$match": {"split": "train"}}, {"$sample": {"size": 10000000}}])
            self.train_cursor = self.collection.find({"split": "train"}).sort("rand_idx", 1)
            self.train_data = MyIterableDataset(self.train_cursor, self.config['buf_size'])

            # self.val_cursor = self.collection.aggregate([{"$match": {"split": "valid"}}, {"$sample": {"size": 10000000}}])

            self.val_cursor = self.collection.find({"split": "valid"}).sort("rand_idx",1)
            self.val_data = MyIterableDataset(self.val_cursor, self.config['buf_size'])


        if stage == "test":
            self.test_cursor = self.collection.find({"split": "test"}).sort("rand_idx", 1)
            self.test_data = MyIterableDataset(self.test_cursor, self.config['buf_size'])

        # if stage == "predict":

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.custom_collate)


    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.custom_collate)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_data, batch_size=self.batch_size, shuffle=False, collate_fn=self.custom_collate)

    # def predict_dataloader(self):


In [ ]:

class LinkData(Data):
    def __init__(self, edge_index_s=None,
                 x_s=None, edge_index_t=None,
                 x_t=None, edge_attr_s=None,
                 edge_attr_t=None,
                 softmax_idx_s=None,
                 softmax_idx_t=None,
                 y=None):

        super().__init__()

        self.edge_index_s = edge_index_s
        self.x_s = x_s

        self.edge_index_t = edge_index_t
        self.x_t = x_t

        self.edge_attr_s = edge_attr_s
        self.edge_attr_t = edge_attr_t

        # softmax index used for AMR model
        self.softmax_idx_s = softmax_idx_s
        self.softmax_idx_t = softmax_idx_t

        self.y = y

    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index_s' or key == 'attention_edge_index_s':
            return self.x_s.size(0)

        elif key == 'edge_index_t' or key == 'attention_edge_index_t':
            return self.x_t.size(0)

        elif key == 'softmax_idx_s':
            return self.softmax_idx_s

        elif key == 'softmax_idx_t':
            return self.softmax_idx_t

        return super().__inc__(key, value, *args, **kwargs)


In [ ]:
pg = PremiseSelectionSeparateGraphs(config = {'buf_size': 128, 'batch_size': 32, 'db_name': 'hol_step','collection_name': 'pretrain_graphs', 'options': ['edge_attr', 'edge_index', 'softmax_idx']})

In [ ]:
pg.setup("fit")

In [ ]:
dl = pg.train_dataloader()

In [ ]:
next(iter(dl))

In [1]:
import torch
from data.utils.dataset import H5DataModule



In [2]:
data_module = H5DataModule(config = {'data_dir':'/home/sean/Documents/phd/repo/aitp/data/utils/processed_data'})


In [3]:
data_module.setup('fit')

In [4]:
from tqdm import tqdm
i = 1
for x in tqdm(data_module.val_dataloader()):
    i += 1
    if i > 3000000:
        print ("looped")
        break


4097it [00:02, 1554.73it/s]
